# Modelo de Deep Learning para la deteccion de fuego(s)

## Modelo basado en redes neuronales convolucionales (CNN) que sigue el paradigma de la visión artificial (Computer Vision) para detectar fuegos en imagenes (fotos, videos).

In [25]:
# Importo las librerías y dependencias necesarias

import tensorflow as tf
import tensorflow_datasets as tfds
import os
import pathlib
import zipfile
import requests
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import pathlib
import zipfile
import gdown
import shutil
import cv2

In [26]:
# Me conecto a mi Google Drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
# Función para descargar el dataset

def download_dataset():
    """
    Descargo y organizo el dataset de Fire Detection from Images desde Google Drive
    """
    dataset_url = "https://drive.google.com/file/d/11UaWR7d8GX43G7ANfPz_CAk80tNQsDF6/view?usp=sharing"
    data_dir = pathlib.Path("fire_detection_dataset")

    if not data_dir.exists():
        print("Descargando dataset...")
        data_dir.mkdir(parents=True, exist_ok=True)
        zip_path = data_dir / "fire_dataset.zip"
        gdown.download(url=dataset_url, output=str(zip_path), quiet=False, fuzzy=True)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        zip_path.unlink()
        source_dir = data_dir / "fire_dataset"
        if source_dir.exists():
            for item in source_dir.iterdir():
                if item.is_dir():
                    dest_path = data_dir / item.name
                    if dest_path.exists():
                        shutil.rmtree(dest_path)
                    shutil.move(str(item), str(data_dir))
            shutil.rmtree(str(source_dir))

    return data_dir


In [28]:
# Función para crear el modelo usando redes neuronales convolucionales (CNN)

def create_model():
    """
    Creo un modelo CNN usando la API secuencial de Keras
    """
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

In [29]:
# Función para preparar los datos

def prepare_data(data_dir, batch_size=32):
    """
    Preparo los generadores de datos para entrenamiento y validación
    """
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )
    train_generator = train_datagen.flow_from_directory(
        str(data_dir), target_size=(224, 224), batch_size=batch_size,
        class_mode='binary', subset='training', classes=['fire_images', 'non_fire_images']
    )
    validation_generator = train_datagen.flow_from_directory(
        str(data_dir), target_size=(224, 224), batch_size=batch_size,
        class_mode='binary', subset='validation', classes=['fire_images', 'non_fire_images']
    )
    return train_generator, validation_generator

In [32]:
# Función para entrenar el modelo

def train_model(model, train_generator, validation_generator, epochs=20):
    """
    Entreno el modelo y guardo el mejor modelo en formato HDF5
    """
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        'fire_detection_model.h5', monitor='val_accuracy', save_best_only=True, mode='max'
    )
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator,
                        callbacks=[checkpoint, early_stopping])
    model.save('fire_detection_model.h5')
    return history

In [33]:
# Función para extraer los frames

def extract_frames(video_path, output_folder, frame_rate=5):
    """
    Extrae frames de un video cada 'frame_rate' fotogramas y los guarda en formato PNG en la carpeta especificada.
    """
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            frame_filename = os.path.join(output_folder, f"frame_{count}.png")
            cv2.imwrite(frame_filename, frame)
        count += 1
    cap.release()
    print(f"Frames guardados en {output_folder}")

In [34]:
# Función para ejecutar el modelo

def main():
    shutil.rmtree("fire_detection_dataset", ignore_errors=True)
    data_dir = download_dataset()
    batch_size, epochs = 32, 5
    model = create_model()
    train_generator, validation_generator = prepare_data(data_dir, batch_size)
    print("Iniciando entrenamiento...")
    history = train_model(model, train_generator, validation_generator, epochs)
    model.save('fire_detection_model.h5')
    print("Modelo guardado como 'fire_detection_model.h5'")
    return model


In [35]:
# Descargo el modelo h5 en local

from google.colab import files

# Descargar el archivo .h5
files.download('fire_detection_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# EJemplo de uso

if __name__ == '__main__':
    main()

video_path = "/content/drive/MyDrive/Dataframes/Noria.mp4"
output_folder = "/content/frames/"
extract_frames(video_path, output_folder)


Descargando dataset...


Downloading...
From (original): https://drive.google.com/uc?id=11UaWR7d8GX43G7ANfPz_CAk80tNQsDF6
From (redirected): https://drive.google.com/uc?id=11UaWR7d8GX43G7ANfPz_CAk80tNQsDF6&confirm=t&uuid=be32210b-e99b-428d-b67d-0eca183f454f
To: /content/fire_detection_dataset/fire_dataset.zip
100%|██████████| 406M/406M [00:09<00:00, 45.1MB/s]


Found 800 images belonging to 2 classes.
Found 199 images belonging to 2 classes.
Iniciando entrenamiento...
Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.7994 - loss: 0.4788

25/25 ━━━━━━━━━━━━━━━━━━━━ 175s 7s/step - accuracy: 0.8019 - loss: 0.4732 - val_accuracy: 0.8543 - val_loss: 0.3044
Epoch 2/5
20/25 ━━━━━━━━━━━━━━━━━━━━ 30s 6s/step - accuracy: 0.9169 - loss: 0.1943